#This is the second notebook where we will use another dataset so that we could use the existing RandomForest Model which is serialized into a pickle file. (Q10)

In [ ]:
#First installing gensim
!pip install gensim

In [ ]:
#These are the libraries we will use here
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
import re
from gensim.models import Word2Vec

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


For this notebook we will use another Kaggle Dataset which is also on "Fake News Detection".
The Link for this - 'https://www.kaggle.com/datasets/mahdimashayekhi/fake-news-detection-dataset?resource=download'

In [ ]:
#Loading the Dataset into a Pandas DataFrame
fake_news_df = pd.read_csv('/content/drive/MyDrive/Classroom/Autumn Internship 2025 Section 1/fake_news_dataset.csv')

In [ ]:
fake_news_df
fake_news_df.head()
fake_news_df.tail()

,title,text,date,source,author,category,label
19995,House party born.,hit and television I change very our happy doo...,2024-12-04,BBC,Gary Miles,Entertainment,fake
19996,Though nation people maybe price box.,fear most meet rock even sea value design stan...,2024-05-26,Daily News,Maria Mcbride,Entertainment,real
19997,Yet exist with experience unit.,activity loss very provide eye west create wha...,2023-04-17,BBC,Kristen Franklin,Entertainment,real
19998,School wide itself item.,term point general common training watch respo...,2024-06-30,Reuters,David Wise,Health,fake
19999,Offer chair cover senior born.,remain pressure glass me six senior though nor...,2024-01-24,Daily News,James Peterson,Health,fake


In [ ]:
#Replacing the labels to 1s and 0s, which will help us use the model
fake_news_df['label'] = fake_news_df['label'].replace({'real': 1, 'fake': 0})
fake_news_df.head()

/tmp/ipython-input-3113067079.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fake_news_df['label'] = fake_news_df['label'].replace({'real': 1, 'fake': 0})


,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,1
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,0
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,0
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,0
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,0


In [ ]:
#Now, we will use the earlier wordopt function and apply that to the text column
def wordopt(text):
  text = text.lower()
  text= re.sub(r'https://\S+|www\.\S+','',text) # remove https:// or www.com
  text = re.sub(r'[^\w]',' ', text) #remove special charecter
  text= re.sub(r'\s+',' ',text) #remove multiple space

  return text

In [ ]:
fake_news_df['text']=fake_news_df['text'].apply(wordopt)
fake_news_df['text'][100]

'challenge include economic place travel media within maintain green even produce decade series business support business worker ten generation front fast weight message task boy join commercial discover light adult keep today note mind forward customer general perform degree size participant prevent government enter benefit color assume describe me service go good never decision spring forward may turn study other staff night medical have prove laugh perhaps relationship relate all then bar next ball official direction heavy fall voice behind company likely culture wait former event know window attack team whole style ago employee push single cost sense ten federal successful table everything despite current word customer short coach general experience prove daughter here would total in culture view in evening approach approach yeah teacher forget research accept loss threat bring maintain then process clear movie bed sea only expert blood total child ahead how stock me keep lose litt

In [ ]:
# Separate independent and target columns from the dataset and stored them in x & y variables
x= fake_news_df['text']
y= fake_news_df['label']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
#This splits the dataframe into the 4 categories

In [ ]:
# import bbc_news data using url link in json format for training Word2Vec word embedding model
news = pd.read_json('https://query.data.world/s/7c6p2lxb3wjibfsfbp4mwy7p7y4y2d?dws=00000')
news_seg = news['content']
print("Size of Word Dictonary for training Word2Vec: ",news_seg.shape[0])
news_seg.head(2)

Size of Word Dictonary for training Word2Vec:  15825


,content
0,The heroin substitute methadone can be used as...
1,The eldest son of North Korean leader Kim Jong...


In [ ]:
# apply wordopt function for text processing
news_seg=news_seg.apply(wordopt)
sentences = [sentence.split() for sentence in news_seg ]
w2v_model = Word2Vec(sentences,  window=5, min_count=5, workers=4)

In [ ]:
# function to convert a sentence into vector form
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [ ]:
# convert train and test dataset into vector form
xv_train = np.array([vectorize(sentence) for sentence in x_train])
xv_test = np.array([vectorize(sentence) for sentence in x_test])

In [ ]:
#Now, importing the existing Random Forest Model here and finding the accuracy of this dataset.
filename2 = '/content/drive/My Drive/Classroom/Autumn Internship 2025 Section 1/word2vec_random_forest_model.pickle'
loaded_model = pickle.load(open(filename2, 'rb'))
result = loaded_model.score(xv_test, y_test)
print(result)

0.4944


#The Accuracy Score is only 0.4944